In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,StructField, StructType
import json

In [2]:
spark = (SparkSession.builder.appName("ResumeProcessing")\
                    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0').getOrCreate())

22/01/12 12:19:31 WARN Utils: Your hostname, djeinaba-HP-Laptop-15-bs1xx resolves to a loopback address: 127.0.0.1; using 10.11.7.230 instead (on interface wlo1)
22/01/12 12:19:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/djeinaba/Bureau/Classroom%20INSA%20EM/Classroom%20S9/Projet_s9/spark/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/djeinaba/.ivy2/cache
The jars for the packages stored in: /home/djeinaba/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-da4f6d19-f44d-4bee-952b-9590a22227c4;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in local-m2-cache
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in local-m2-cache
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in local-m2-cache
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found co

In [3]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "indeedData") \
    .option("startingOffsets","earliest") \
    .load()

In [4]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
@udf(returnType=StringType())
def jsonparse(column, name):
    return str(json.loads(column)[name])


    

In [6]:
lines = df\
    .selectExpr("CAST(value AS STRING) as json")\
    .withColumn("companies", jsonparse(col("json"), lit("companies")))\
    .withColumn("job_title", jsonparse(col("json"), lit("job title")))\
    .withColumn("job_type", jsonparse(col("json"), lit("job Type")))\
    .withColumn("job Description", jsonparse(col("json"), lit("job Description")))\
    .withColumn("job link", jsonparse(col("json"), lit("job link")))\
    .select("companies", "job_title", "job_type","job Description","job link")


In [24]:
lines.printSchema()


root
 |-- companies: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_type: string (nullable = true)
 |-- job Description: string (nullable = true)
 |-- job link: string (nullable = true)



In [ ]:
data_agg_write_stream = lines \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("jobPostTable") \
        .start()

data_agg_write_stream.awaitTermination(1)

In [26]:
jobPosting_df = spark.sql('SELECT * FROM JobPostTable')

In [27]:
jobPosting_df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           companies|           job_title|            job_type|     job Description|            job link|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  {}|                  {}|                  {}|                  {}|                  {}|
|{'0': 'Google', '...|{'0': 'Data Scien...|{'0': 'None', '1'...|{'0': "Note: By a...|{'0': 'https://ww...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [17]:
jobPosting_df.printSchema()

root
 |-- companies: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_type: string (nullable = true)
 |-- job Description: string (nullable = true)
 |-- job link: string (nullable = true)



In [18]:
type(jobPosting_df)

pyspark.sql.dataframe.DataFrame

In [28]:
from pyspark.sql.functions import udf, explode
import json

@udf("map<string, string>")
def parse(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        pass 

In [29]:
jobPosting_df.select(explode(parse("companies")).alias("companies_name", "index")).show()

+--------------+-----+
|companies_name|index|
+--------------+-----+
+--------------+-----+



In [ ]:
import pyspark.sql.functions as F
schema=jobPosting_df.select(F.schema_of_json(jobPosting_df.select("jobposttable.companies").first()[0])).first()[0]

In [ ]:
schema =StructType([StructField("companies", StringType(),True),
                           StructField("job_title", StringType(),True),
                           StructField("job_type",StringType(),True),
                           StructField("job Description", StringType(),True),
                           StructField("job link", StringType(),True),
                           ])

In [ ]:
df_1=jobPosting_df.withColumn("companies", F.from_json("companies", schema))\
                  .withColumn("job_title", F.from_json("job_title", schema))\
                  .withColumn("job_type", F.from_json("job_type", schema))\
                  .withColumn("job Description", F.from_json("job Description", schema))\
                  .withColumn("job link", F.from_json("job link", schema))\
                  .select("*")


In [ ]:
df_1.show()